In [9]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


# _set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

In [10]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Plan-and-execute"

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]

In [12]:
from langchain import hub
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("wfh/react-agent-executor")
prompt.pretty_print()

# Choose the LLM that will drive the agent
# llm = ChatOpenAI(model="gpt-4-turbo-preview")
llm = ChatOpenAI(
    model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    base_url="http://10.2.125.37:1234/v1",
    api_key="lm-studio"
) 
agent_executor = create_react_agent(llm, tools, messages_modifier=prompt)

================================ System Message ================================

You are a helpful assistant.

============================= Messages Placeholder =============================

{{messages}}


/tmp/ipykernel_64578/2149119283.py:17: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.2.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, tools, messages_modifier=prompt)


In [13]:
agent_executor.invoke({"messages": [("user", "who is the winnner of the us open")]})

{'messages': [HumanMessage(content='who is the winnner of the us open', id='2646b006-5cf4-4567-b05c-519c0efd4e65'),
  AIMessage(content='However, I\'m a large language model, I don\'t have real-time access to current information. But I can suggest some possible ways to find out who won the US Open:\n\n1. Check the official US Open website (usopen.org) for the latest news and results.\n2. Look up sports websites such as ESPN, CNN Sports, or Fox Sports for updates on the tournament.\n3. Search online for "US Open winner" along with the current year to find relevant articles and reports.\n\nIf you provide me with a specific year, I can try to help you find out who won the US Open for that particular year!\n\nWould you like me to look up a specific year?', response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 31, 'total_tokens': 169}, 'model_name': 'lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf', 'system_fingerprint': None, 